# Interactions - Lab

## Introduction

In this lab, you'll explore interactions in the Ames Housing dataset.

## Objectives

You will be able to:
- Implement interaction terms in Python using the `sklearn` and `statsmodels` packages 
- Interpret interaction variables in the context of a real-world problem 

## Build a baseline model 

You'll use a couple of built-in functions, which we imported for you below: 

In [66]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

If you still want to build a model in the end, you can do that, but this lab will just focus on finding meaningful insights in interactions and how they can improve $R^2$ values.

In [2]:
regression = LinearRegression()

Create a baseline model which includes all the variables we selected from the Ames housing data set to predict the house prices. Then use 10-fold cross-validation and report the mean $R^2$ value as the baseline $R^2$.

In [137]:
ames = pd.read_csv('ames.csv')
new_ames=pd.DataFrame()
data_fin = pd.DataFrame()


continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

def scale(df):
    return (df-np.mean(df))/np.std(df)


## New_names
continuous_log_names = ["Log_"+name for name in continuous]

# transform of continuous data
new_ames[continuous_log_names] = np.log(ames[continuous])


# One hot encode of categorical data
encoder = OneHotEncoder(handle_unknown = "ignore")
labelencoder = LabelEncoder()
encoder.fit(ames[categoricals])
new_cat = encoder.transform(ames[categoricals])
new_cat.todense()
encoder.get_feature_names()
New_cat_data = pd.DataFrame(new_cat.todense(), columns=encoder.get_feature_names())


data_fin = pd.concat([new_ames[continuous_log_names].apply(scale), New_cat_data], axis =1 )

data_fin

X = data_fin.drop("Log_SalePrice", axis = 1)
y = data_fin["Log_SalePrice"] 


linreg=LinearRegression()
crossvalidation = KFold(n_splits=10, shuffle=True, random_state=1)
baseline = np.mean(cross_val_score(linreg, X, y, scoring="r2" , cv = crossvalidation))
baseline



0.8174066703404836

In [106]:
New_cat_data

,x0_1Fam,x0_2fmCon,x0_Duplex,x0_Twnhs,x0_TwnhsE,x1_Ex,x1_Fa,x1_Gd,x1_TA,x2_COD,...,x5_NoRidge,x5_NridgHt,x5_OldTown,x5_SWISU,x5_Sawyer,x5_SawyerW,x5_Somerst,x5_StoneBr,x5_Timber,x5_Veenker
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1456,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1457,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1458,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## See how interactions improve your baseline

Next, create all possible combinations of interactions, loop over them and add them to the baseline model one by one to see how they affect the $R^2$. We'll look at the 3 interactions which have the biggest effect on our $R^2$, so print out the top 3 combinations.

You will create a `for` loop to loop through all the combinations of 2 predictors. You can use `combinations` from itertools to create a list of all the pairwise combinations. To find more info on how this is done, have a look [here](https://docs.python.org/2/library/itertools.html).

Since there are so many different neighbourhoods we will exclude

In [156]:
from itertools import combinations
combinat = combinations(X.columns, 2)
data = X.copy()
count = 0
combination_list=[]

for (a, b) in combinat:
    data["new_feature"] = data[a]*data[b]
    crossvalidation = KFold(n_splits=10, shuffle=True, random_state=1)
    newline = np.mean(cross_val_score(linreg, data, y, scoring="r2" , cv = crossvalidation))
    if newline > baseline:
        combination_list.append((a, b, newline))
    if count % 100 == 0:
        print(count, "combinations")
    count +=1



0 combinations
100 combinations
200 combinations
300 combinations
400 combinations
500 combinations
600 combinations
700 combinations
800 combinations
900 combinations
1000 combinations
1100 combinations
1200 combinations
1300 combinations


In [171]:
new = sorted(combination_list, key = lambda inter: inter[2], reverse = True)[:5]
new

[('x1_Ex', 'x5_ClearCr', 0.8203128237536019),
 ('x0_Duplex', 'x4_Pave', 0.8196563892558343),
 ('x0_Duplex', 'x4_Grvl', 0.8196501874986861),
 ('Log_LotArea', 'x5_Mitchel', 0.8196372136006372),
 ('Log_GrLivArea', 'x1_TA', 0.8196235715989975)]

In [165]:
new
# code to find top interactions by R^2 value here

It looks like the top interactions involve the Neighborhood_Edwards feature so lets add the interaction between LotArea and Edwards to our model.

We can interpret this feature as the relationship between LotArea and SalePrice when the house is in Edwards or not.

## Visualize the Interaction

Separate all houses that are located in Edwards and those that are not. Run a linear regression on each population against `SalePrice`. Visualize the regression line and data points with price on the y axis and LotArea on the x axis.

In [ ]:
# Visualization code here

## Build a final model with interactions

Use 10-fold cross-validation to build a model using the above interaction. 

In [ ]:
# code here

Our $R^2$ has increased considerably! Let's have a look in `statsmodels` to see if this interactions are significant.

In [ ]:
# code here

What is your conclusion here?

In [ ]:
# formulate your conclusion

## Summary

You should now understand how to include interaction effects in your model! As you can see, interactions can have a strong impact on linear regression models, and they should always be considered when you are constructing your models.